<a href="https://colab.research.google.com/github/wileyw/DeepLearningDemos/blob/master/SiameseNetwork/siamese_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Siamese Network